## cos_simで出した候補研究者の業績データを作成

In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None
import pickle
from tqdm import tqdm
import seaborn as sns
import os, sys, gc, random, itertools, math, pickle
from itertools import combinations
import networkx as nx


In [85]:
authid_12 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/2021/A1.2_who_guideline_development_group.npy')
authid_13 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/A1.3_who_systematic_review_team_revise.npy')
authid_14 = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/2021/A1.4_who_external_methodologists.npy')
authid_15 = np.load('//disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/member_data/A1.5_who_external_review_team_revise.npy')

boradmember_authid = list(authid_12)+list(authid_13)+list(authid_14)+list(authid_15)

authid_list1 = [35780677200 if x == 7006806977 else 57206422848 if x == 54791745100  else 57219144066 if x == 55588756600  else 57219144066 if x == 57191727610  else 57219144066 if x == 14037485900 else x for x in boradmember_authid]
treatment_authids = list(set(authid_list1))

In [86]:
len(treatment_authids)

108

In [14]:
top_k_similar_authids = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/top_k_similar_authids.npy', allow_pickle=True)
top_k_similar_authids = top_k_similar_authids.item()

In [15]:
top_k_similar_authids

{55190092800: [56641796800,
  17343447800,
  6604023299,
  25960839100,
  7102151876,
  6701815935,
  55197325500,
  8682473200,
  6603854659,
  57191400038,
  57208839454,
  57206617186,
  8337745000,
  36769118100,
  55472005900,
  35592531500,
  57188675726,
  57202148134,
  7006946563,
  7202522962,
  7004869473,
  34770698800,
  7102484117,
  55568524311,
  57210951407,
  7004063323,
  7006104636,
  7006331532,
  16228572800,
  6602151433,
  55629754700,
  35619411000,
  7006557124,
  7102576614,
  6603162449,
  7501691986,
  6507633685,
  6602851058,
  7402450845,
  7005065089],
 57191900162: [35077577400,
  55915457700,
  36704098200,
  56324482500,
  56324030000,
  56324314100,
  35725862900,
  36466375500,
  24821710300,
  6506880484,
  26643853000,
  6602844186,
  55778241100,
  58397721100,
  55062714000,
  57196523112,
  6701515513,
  7004426039,
  56263115600,
  7004848381,
  35075257000,
  55913573100,
  55912492600,
  55913458700,
  6603828606,
  56414745100,
  558951895

In [21]:
top_k_similar_authids_ls = []

key_ls = top_k_similar_authids.keys()
value_ls = top_k_similar_authids.values()


top_k_similar_authids_ls += list(key_ls)
for i in value_ls:
    top_k_similar_authids_ls += i

target_authids_set = set(top_k_similar_authids_ls)

In [22]:
len(target_authids_set)

2747

In [23]:
average_researcher_vectors = np.load('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/researcher_average_embedding_matrix_2011_15.npy')


In [24]:
average_researcher_vectors.shape

(11289655, 1536)

In [25]:
average_researcher_vectors_df = pd.DataFrame(average_researcher_vectors)

In [26]:
average_researcher_vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,0.002669,0.016875,0.039188,0.043955,-0.013159,-0.010910,0.007247,0.019239,0.009222,0.001308,...,-0.002898,0.003575,-0.006216,0.000681,-0.021566,-0.012915,0.004388,-0.002048,-0.011570,-0.003535
1,0.028011,0.027607,0.012062,0.039757,0.007704,-0.043895,0.010468,0.019665,-0.015682,0.005766,...,-0.012377,-0.003302,-0.009844,0.005062,-0.031792,0.024563,0.002708,-0.006634,0.005638,-0.001890
2,-0.010297,0.028573,-0.018459,0.012180,0.015227,0.044324,0.017603,0.028985,-0.027727,0.011111,...,-0.020953,0.015231,-0.026393,0.003340,-0.006521,-0.019538,0.049663,-0.025146,-0.024796,-0.016072
3,0.019268,0.031133,0.009725,-0.011852,0.020986,0.005772,0.004396,-0.018396,0.015951,0.012666,...,-0.000722,0.022226,-0.032007,-0.003730,-0.016344,0.004389,0.009063,-0.012784,0.006892,0.015211
4,-0.007302,0.006359,0.048152,0.021727,-0.009678,0.008319,0.017433,-0.004672,0.008912,0.008910,...,-0.010200,0.006197,-0.029086,-0.007651,-0.015330,-0.015825,0.015266,0.008702,-0.016099,0.016176


In [27]:

# 保存先のファイルパス（拡張子は.pklなど任意に設定してください）
file_path = f'/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/sparse_one_hot_authids_idx.pkl'

# バイナリ書き込みモードでファイルを開いて保存
with open(file_path, 'rb') as f:
    authid_to_idx = pickle.load(f)

In [28]:
average_researcher_vectors_df['authids'] = authid_to_idx.keys()

In [29]:
average_researcher_vectors_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,authids
0,0.002669,0.016875,0.039188,0.043955,-0.013159,-0.010910,0.007247,0.019239,0.009222,0.001308,...,0.003575,-0.006216,0.000681,-0.021566,-0.012915,0.004388,-0.002048,-0.011570,-0.003535,1
1,0.028011,0.027607,0.012062,0.039757,0.007704,-0.043895,0.010468,0.019665,-0.015682,0.005766,...,-0.003302,-0.009844,0.005062,-0.031792,0.024563,0.002708,-0.006634,0.005638,-0.001890,2
2,-0.010297,0.028573,-0.018459,0.012180,0.015227,0.044324,0.017603,0.028985,-0.027727,0.011111,...,0.015231,-0.026393,0.003340,-0.006521,-0.019538,0.049663,-0.025146,-0.024796,-0.016072,4
3,0.019268,0.031133,0.009725,-0.011852,0.020986,0.005772,0.004396,-0.018396,0.015951,0.012666,...,0.022226,-0.032007,-0.003730,-0.016344,0.004389,0.009063,-0.012784,0.006892,0.015211,6
4,-0.007302,0.006359,0.048152,0.021727,-0.009678,0.008319,0.017433,-0.004672,0.008912,0.008910,...,0.006197,-0.029086,-0.007651,-0.015330,-0.015825,0.015266,0.008702,-0.016099,0.016176,8


In [30]:
average_researcher_vectors_df = average_researcher_vectors_df[average_researcher_vectors_df['authids']!=2]

In [31]:
author_num_cited = pd.read_pickle("/disks/qnap3/shared/scopus-24/data/author/year_num_cited.pickle")

In [32]:
author_num_cited.reset_index(inplace=True)

In [33]:
author_num_cited.rename(columns={'references_author':'authids'},inplace=True)

In [34]:
author_num_cited.head()

citedby_year,authids,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0
0,1,1.0,12.0,22.0,32.0,56.0,58.0,56.0,83.0,79.0,...,596.0,590.0,668.0,701.0,628.0,705.0,747.0,709.0,780.0,NaN
1,2,NaN,6.0,1.0,1.0,1.0,1.0,7.0,9.0,10.0,...,116.0,120.0,109.0,98.0,88.0,84.0,101.0,91.0,67.0,NaN
2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,34.0,39.0,21.0,22.0,25.0,17.0,24.0,12.0,21.0,NaN
3,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,3.0,NaN
4,8,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.0,...,120.0,180.0,232.0,421.0,749.0,1354.0,2051.0,2065.0,2108.0,1.0


In [35]:
author_num_cited.set_index('authids',inplace=True)

In [36]:
author_num_cited.head()

citedby_year,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0
authids,,,,,,,,,,,,,,,,,,,,,
1,1.0,12.0,22.0,32.0,56.0,58.0,56.0,83.0,79.0,69.0,...,596.0,590.0,668.0,701.0,628.0,705.0,747.0,709.0,780.0,NaN
2,NaN,6.0,1.0,1.0,1.0,1.0,7.0,9.0,10.0,55.0,...,116.0,120.0,109.0,98.0,88.0,84.0,101.0,91.0,67.0,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,...,34.0,39.0,21.0,22.0,25.0,17.0,24.0,12.0,21.0,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,1.0,3.0,2.0,2.0,2.0,3.0,2.0,3.0,NaN
8,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,1.0,1.0,...,120.0,180.0,232.0,421.0,749.0,1354.0,2051.0,2065.0,2108.0,1.0


In [37]:
author_num_cited = author_num_cited.fillna(0)

In [38]:
author_num_cited_cum = author_num_cited.T.cumsum().T

In [39]:
author_num_cited_cum.head()

citedby_year,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0
authids,,,,,,,,,,,,,,,,,,,,,
1,1.0,13.0,35.0,67.0,123.0,181.0,237.0,320.0,399.0,468.0,...,15542.0,16132.0,16800.0,17501.0,18129.0,18834.0,19581.0,20290.0,21070.0,21070.0
2,0.0,6.0,7.0,8.0,9.0,10.0,17.0,26.0,36.0,91.0,...,5045.0,5165.0,5274.0,5372.0,5460.0,5544.0,5645.0,5736.0,5803.0,5803.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,...,1009.0,1048.0,1069.0,1091.0,1116.0,1133.0,1157.0,1169.0,1190.0,1190.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,4.0,7.0,9.0,11.0,13.0,16.0,18.0,21.0,21.0
8,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,4.0,5.0,...,957.0,1137.0,1369.0,1790.0,2539.0,3893.0,5944.0,8009.0,10117.0,10118.0


In [40]:
author_num_cited_cum.reset_index(inplace=True)

In [41]:
author_num_cited_cum.head()

citedby_year,authids,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,...,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0
0,1,1.0,13.0,35.0,67.0,123.0,181.0,237.0,320.0,399.0,...,15542.0,16132.0,16800.0,17501.0,18129.0,18834.0,19581.0,20290.0,21070.0,21070.0
1,2,0.0,6.0,7.0,8.0,9.0,10.0,17.0,26.0,36.0,...,5045.0,5165.0,5274.0,5372.0,5460.0,5544.0,5645.0,5736.0,5803.0,5803.0
2,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1009.0,1048.0,1069.0,1091.0,1116.0,1133.0,1157.0,1169.0,1190.0,1190.0
3,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,4.0,7.0,9.0,11.0,13.0,16.0,18.0,21.0,21.0
4,8,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,4.0,...,957.0,1137.0,1369.0,1790.0,2539.0,3893.0,5944.0,8009.0,10117.0,10118.0


In [116]:
author_num_cited_cum1 = author_num_cited_cum[['authids',2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]]

In [117]:
target_years = range(2011, 2024)

# Update column names
author_num_cited_cum1.rename(
    columns={
        col: f"academic_citation_{int(col)}"
        for col in author_num_cited_cum1.columns
        if isinstance(col, float) and int(col) in target_years
    },
    inplace=True
)

/tmp/ipykernel_588919/3295752475.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_num_cited_cum1.rename(


In [118]:
author_num_cited_cum1.head()

citedby_year,authids,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,academic_citation_2020,academic_citation_2021,academic_citation_2022,academic_citation_2023
0,1,13130.0,13725.0,14372.0,14946.0,15542.0,16132.0,16800.0,17501.0,18129.0,18834.0,19581.0,20290.0,21070.0
1,2,4458.0,4619.0,4773.0,4929.0,5045.0,5165.0,5274.0,5372.0,5460.0,5544.0,5645.0,5736.0,5803.0
2,4,870.0,905.0,942.0,975.0,1009.0,1048.0,1069.0,1091.0,1116.0,1133.0,1157.0,1169.0,1190.0
3,6,1.0,1.0,1.0,1.0,3.0,4.0,7.0,9.0,11.0,13.0,16.0,18.0,21.0
4,8,547.0,636.0,737.0,837.0,957.0,1137.0,1369.0,1790.0,2539.0,3893.0,5944.0,8009.0,10117.0


In [119]:
author_num_cited_cum1.shape

(36750640, 14)

In [120]:
author_num_cited_cum4

citedby_year,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,academic_citation_2020,academic_citation_2021,academic_citation_2022
authids,,,,,,,,,,,,
6506139146,120.0,124.0,144.0,166.0,181.0,201.0,214.0,229.0,246.0,259.0,275.0,293.0
6506306535,157.0,204.0,272.0,340.0,438.0,574.0,746.0,914.0,1116.0,1331.0,1565.0,1820.0
6507352648,266.0,380.0,526.0,698.0,948.0,1116.0,1308.0,1574.0,1865.0,2242.0,2728.0,3231.0
6507497656,212.0,261.0,302.0,332.0,362.0,404.0,447.0,515.0,604.0,733.0,919.0,1173.0
6507813087,1040.0,1292.0,1515.0,1789.0,2086.0,2316.0,2545.0,2803.0,3071.0,3437.0,3939.0,4555.0
...,...,...,...,...,...,...,...,...,...,...,...,...
57217208060,11702.0,13568.0,15688.0,18186.0,20995.0,24375.0,28440.0,33283.0,39171.0,45851.0,53270.0,60935.0
57218330976,3828.0,4594.0,6007.0,8054.0,11029.0,15573.0,21510.0,28937.0,37833.0,48255.0,61741.0,79210.0
57219144066,2841.0,3207.0,3537.0,4006.0,4532.0,5280.0,6064.0,7039.0,8315.0,10322.0,13306.0,16439.0


In [121]:
author_num_cited_cum1['authids'] = author_num_cited_cum1['authids'].replace({7006806977: 35780677200, 54791745100: 57206422848, 55588756600:57219144066, 57191727610:57219144066, 14037485900:57219144066})

           

/tmp/ipykernel_588919/2057405878.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_num_cited_cum1['authids'] = author_num_cited_cum1['authids'].replace({7006806977: 35780677200, 54791745100: 57206422848, 55588756600:57219144066, 57191727610:57219144066, 14037485900:57219144066})


In [125]:
author_num_cited_cum2 = author_num_cited_cum1.groupby('authids').sum()[['academic_citation_2011', 'academic_citation_2012', 'academic_citation_2013', 'academic_citation_2014', 'academic_citation_2015', 'academic_citation_2016', 'academic_citation_2017', 'academic_citation_2018', 'academic_citation_2019', 'academic_citation_2020', 'academic_citation_2021', 'academic_citation_2022', 'academic_citation_2023']]








In [126]:
author_num_cited_cum2.shape

(36750635, 13)

In [127]:
author_num_cited_cum2.head()

citedby_year,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,academic_citation_2020,academic_citation_2021,academic_citation_2022,academic_citation_2023
authids,,,,,,,,,,,,,
1,13130.0,13725.0,14372.0,14946.0,15542.0,16132.0,16800.0,17501.0,18129.0,18834.0,19581.0,20290.0,21070.0
2,4458.0,4619.0,4773.0,4929.0,5045.0,5165.0,5274.0,5372.0,5460.0,5544.0,5645.0,5736.0,5803.0
4,870.0,905.0,942.0,975.0,1009.0,1048.0,1069.0,1091.0,1116.0,1133.0,1157.0,1169.0,1190.0
6,1.0,1.0,1.0,1.0,3.0,4.0,7.0,9.0,11.0,13.0,16.0,18.0,21.0
8,547.0,636.0,737.0,837.0,957.0,1137.0,1369.0,1790.0,2539.0,3893.0,5944.0,8009.0,10117.0


In [128]:
author_num_cited_cum4 = author_num_cited_cum2[author_num_cited_cum2.index.isin(treatment_authids)]
author_num_cited_cum4

citedby_year,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,academic_citation_2020,academic_citation_2021,academic_citation_2022,academic_citation_2023
authids,,,,,,,,,,,,,
6506139146,120.0,124.0,144.0,166.0,181.0,201.0,214.0,229.0,246.0,259.0,275.0,293.0,306.0
6506306535,157.0,204.0,272.0,340.0,438.0,574.0,746.0,914.0,1116.0,1331.0,1565.0,1820.0,2077.0
6507352648,266.0,380.0,526.0,698.0,948.0,1116.0,1308.0,1574.0,1865.0,2242.0,2728.0,3231.0,3701.0
6507497656,212.0,261.0,302.0,332.0,362.0,404.0,447.0,515.0,604.0,733.0,919.0,1173.0,1498.0
6507813087,1040.0,1292.0,1515.0,1789.0,2086.0,2316.0,2545.0,2803.0,3071.0,3437.0,3939.0,4555.0,5085.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57217208060,11702.0,13568.0,15688.0,18186.0,20995.0,24375.0,28440.0,33283.0,39171.0,45851.0,53270.0,60935.0,67777.0
57218330976,3828.0,4594.0,6007.0,8054.0,11029.0,15573.0,21510.0,28937.0,37833.0,48255.0,61741.0,79210.0,98795.0
57219144066,2841.0,3207.0,3537.0,4006.0,4532.0,5280.0,6064.0,7039.0,8315.0,10322.0,13306.0,16439.0,19577.0


### 政策引用数を集計

In [53]:
ov_yearly_df = pd.read_pickle('/disks/qnap3/shared/Overton/scopus24_yearly_IGO_citation.pickle')

In [54]:
ov_yearly_df.head()

year,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
eid,,,,,,,,,,,,,,,,,,,,,
196.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
353.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
641.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
874.0,0,0,0,0,0,0,0,0,0,0,...,0,1,2,2,1,0,0,0,0,0
1040.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [55]:
ov_yearly_df = ov_yearly_df.T.cumsum().T

In [56]:
ov_yearly_df.reset_index(inplace=True)

In [166]:
ov_yearly_df1 = ov_yearly_df[['eid', 2011,2012,2013,2014,2015,2016,2017, 2018, 2019, 2020, 2021, 2022,2023]]

In [167]:
ov_yearly_df1.rename(columns={2011:'policy_citation_2011',2012:'policy_citation_2012',2013:'policy_citation_2013',2014:'policy_citation_2014',2015:'policy_citation_2015',2016:'policy_citation_2016', 2017:'policy_citation_2017', 2018:'policy_citation_2018', 2019:'policy_citation_2019', 2020:'policy_citation_2020', 2021:'policy_citation_2021',2022:'policy_citation_2022',2023:'policy_citation_2023'},inplace=True)

/tmp/ipykernel_588919/2856930980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ov_yearly_df1.rename(columns={2011:'policy_citation_2011',2012:'policy_citation_2012',2013:'policy_citation_2013',2014:'policy_citation_2014',2015:'policy_citation_2015',2016:'policy_citation_2016', 2017:'policy_citation_2017', 2018:'policy_citation_2018', 2019:'policy_citation_2019', 2020:'policy_citation_2020', 2021:'policy_citation_2021',2022:'policy_citation_2022',2023:'policy_citation_2023'},inplace=True)


In [168]:
ov_yearly_df1.head()

year,eid,policy_citation_2011,policy_citation_2012,policy_citation_2013,policy_citation_2014,policy_citation_2015,policy_citation_2016,policy_citation_2017,policy_citation_2018,policy_citation_2019,policy_citation_2020,policy_citation_2021,policy_citation_2022,policy_citation_2023
0,196.0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,353.0,3,3,3,3,3,3,3,3,3,3,3,3,3
2,641.0,4,4,4,6,6,6,6,6,7,7,8,8,8
3,874.0,6,9,11,13,13,14,16,18,19,19,19,19,19
4,1040.0,0,0,0,0,0,0,0,1,1,1,1,1,1


In [169]:
ov_yearly_df1 = ov_yearly_df1.fillna(0)

In [135]:
authids_df = pd.read_pickle('/disks/qnap3/shared/scopus-24/data/paper/authids.pickle')
authids_df = pd.DataFrame(authids_df)
authids_df.reset_index(inplace=True)
authids_df.rename(columns={'index':'eid'},inplace=True)

In [136]:
authids_df.head()

,eid,authids
0,107,"[7004442182, 7004442182, 6603403884, 57203215073, 6602418637]"
1,110,"[7006050637, 6603697521, 6602703231]"
2,116,"[57218153744, 7402496690, 35351983100]"
3,117,"[7005148342, 7003508994]"
4,118,"[6603815313, 7004562485]"


In [137]:
authids_explode_df = authids_df.explode('authids')

In [138]:
authids_explode_df.head()

,eid,authids
0,107,7004442182
0,107,7004442182
0,107,6603403884
0,107,57203215073
0,107,6602418637


In [159]:
authids_explode_df2 = authids_explode_df[authids_explode_df['eid'].isin(ov_yearly_df1['eid'].values)]

In [170]:
ov_yearly_citation_df = pd.merge(authids_explode_df2, ov_yearly_df1, on='eid', how='left')

In [171]:
ov_yearly_citation_df = ov_yearly_citation_df[~ov_yearly_citation_df['authids'].isnull()]

In [172]:
ov_yearly_citation_df.head()

,eid,authids,policy_citation_2011,policy_citation_2012,policy_citation_2013,policy_citation_2014,policy_citation_2015,policy_citation_2016,policy_citation_2017,policy_citation_2018,policy_citation_2019,policy_citation_2020,policy_citation_2021,policy_citation_2022,policy_citation_2023
0,196,7101710588,0,0,0,0,0,0,0,0,0,0,0,0,1
1,353,6508316485,3,3,3,3,3,3,3,3,3,3,3,3,3
2,353,7003851430,3,3,3,3,3,3,3,3,3,3,3,3,3
3,353,7004921497,3,3,3,3,3,3,3,3,3,3,3,3,3
4,353,7402306805,3,3,3,3,3,3,3,3,3,3,3,3,3


In [173]:
ov_yearly_citation_df['authids'] = ov_yearly_citation_df['authids'].replace({7006806977: 35780677200, 54791745100: 57206422848, 55588756600:57219144066, 57191727610:57219144066, 14037485900:57219144066})


/tmp/ipykernel_588919/390660526.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ov_yearly_citation_df['authids'] = ov_yearly_citation_df['authids'].replace({7006806977: 35780677200, 54791745100: 57206422848, 55588756600:57219144066, 57191727610:57219144066, 14037485900:57219144066})


In [174]:
ov_yearly_citation_df = ov_yearly_citation_df[['authids','policy_citation_2011','policy_citation_2012','policy_citation_2013','policy_citation_2014','policy_citation_2015','policy_citation_2016','policy_citation_2017','policy_citation_2018','policy_citation_2019','policy_citation_2020', 'policy_citation_2021','policy_citation_2022','policy_citation_2023']]

In [178]:
ov_yearly_citation_df1 = ov_yearly_citation_df.groupby('authids').sum()[['policy_citation_2011','policy_citation_2012','policy_citation_2013','policy_citation_2014','policy_citation_2015','policy_citation_2016','policy_citation_2017','policy_citation_2018','policy_citation_2019','policy_citation_2020', 'policy_citation_2021','policy_citation_2022','policy_citation_2023']]

In [179]:
ov_yearly_citation_df1.shape

(1625499, 13)

In [180]:
ov_yearly_citation_df1.head()

,policy_citation_2011,policy_citation_2012,policy_citation_2013,policy_citation_2014,policy_citation_2015,policy_citation_2016,policy_citation_2017,policy_citation_2018,policy_citation_2019,policy_citation_2020,policy_citation_2021,policy_citation_2022,policy_citation_2023
authids,,,,,,,,,,,,,
0,3956,5478,6823,8171,9531,11072,12324,13771,15425,16730,18445,20163,21515
1,6,6,8,8,8,8,8,8,8,8,8,8,8
8,1,5,5,5,5,5,5,11,11,11,12,12,12
6503845504,0,2,2,2,2,2,2,2,2,2,2,2,2
6503845527,9,11,14,15,15,16,16,17,18,19,19,22,22


In [181]:
ov_yearly_citation_df2 = ov_yearly_citation_df1[ov_yearly_citation_df1.index.isin(treatment_authids)]

# treatment_authids

In [182]:
len(target_authids_set)

2747

In [183]:
treatment_authids

[55190092800,
 57191900162,
 36125281800,
 6506139146,
 7406531615,
 7004532767,
 7402105900,
 14825606700,
 7103344174,
 57494449200,
 7003326006,
 6602905655,
 6701458488,
 35379363900,
 7102270013,
 26666591300,
 6507352648,
 26424253000,
 6603575372,
 7004773965,
 6701815372,
 35780677200,
 55263109200,
 13608072800,
 7007123042,
 24177416300,
 7410184308,
 57190068857,
 57203088526,
 7004818581,
 22135643800,
 55245988000,
 57223422630,
 7202296490,
 12783574700,
 7101602991,
 35396805300,
 17035269300,
 36128837300,
 6508112567,
 6507497656,
 55981522100,
 57197802175,
 16205094600,
 35168055500,
 56553873100,
 55232554700,
 7403393231,
 15064852700,
 6507813087,
 24474239200,
 15752092900,
 7007103216,
 57217208060,
 7402866430,
 7102579456,
 6602157313,
 57006461700,
 7003735816,
 57207742220,
 56623591700,
 56006575900,
 6603940133,
 36705318700,
 57197110584,
 57201178426,
 7003533118,
 56920788800,
 7402372928,
 55220462400,
 57206422848,
 34568758600,
 55632925000,
 2477346

In [184]:
ov_yearly_citation_df2.shape

(100, 13)

### 論文本数

In [92]:
author_paperlist = pd.read_pickle('/disks/qnap3/shared/scopus-24/data/author/year_productivity.pickle')

In [93]:
author_paperlist.head()

year,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
authids,,,,,,,,,,,,,,,,,,,,,
1,2097.0,1740.0,939.0,1597.0,2168.0,2139.0,2389.0,2220.0,2309.0,2255.0,...,7.0,19.0,11.0,9.0,14.0,10.0,9.0,91.0,9.0,0.0
2,0.0,2.0,1.0,1.0,0.0,3.0,1.0,0.0,2.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57042534400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0
57982058500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0


In [94]:
author_paperlist = author_paperlist.T.cumsum().T

In [95]:
author_paperlist.head()

year,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
authids,,,,,,,,,,,,,,,,,,,,,
1,2097.0,3837.0,4776.0,6373.0,8541.0,10680.0,13069.0,15289.0,17598.0,19853.0,...,70635.0,70654.0,70665.0,70674.0,70688.0,70698.0,70707.0,70798.0,70807.0,70807.0
2,0.0,2.0,3.0,4.0,4.0,7.0,8.0,8.0,10.0,12.0,...,256.0,256.0,256.0,256.0,256.0,256.0,256.0,256.0,256.0,256.0
57042534400,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,18.0,18.0,19.0,20.0,20.0,20.0,22.0,22.0,22.0,22.0
57982058500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,3.0


In [96]:
author_paperlist1 = author_paperlist[[2015]]

In [97]:
author_paperlist1.rename(columns={2015:'paper_num_2015'},inplace=True)

/tmp/ipykernel_588919/775144272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  author_paperlist1.rename(columns={2015:'paper_num_2015'},inplace=True)


In [98]:
author_paperlist1.head()

year,paper_num_2015
authids,
1,70635.0
2,256.0
57042534400,2.0
4,18.0
57982058500,0.0


In [15]:
# target_embedding_explode_df1 = pd.merge(target_embedding_explode_df, author_paperlist1, left_index=True, right_index=True, how='left')

## 情報を統合

In [99]:
target_author_paperlist = author_paperlist1[author_paperlist1.index.isin(target_authids_set)]

In [100]:
target_author_paperlist

year,paper_num_2015
authids,
11140076200,38.0
8858824900,32.0
55835033800,4.0
21743832000,27.0
36642045300,50.0
...,...
23487676500,114.0
23487806800,53.0
56371163300,93.0


In [151]:
target_author_num_cited = author_num_cited_cum2[author_num_cited_cum2.index.isin(target_authids_set)]
target_author_num_cited

citedby_year,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,academic_citation_2020,academic_citation_2021,academic_citation_2022,academic_citation_2023
authids,,,,,,,,,,,,,
6503956214,333.0,493.0,692.0,1053.0,1398.0,1850.0,2307.0,2732.0,3138.0,3515.0,3818.0,4091.0,4344.0
6503980107,45.0,58.0,74.0,90.0,108.0,125.0,142.0,171.0,207.0,243.0,288.0,322.0,365.0
6504250744,46.0,127.0,317.0,737.0,1306.0,2020.0,2891.0,3897.0,5052.0,6327.0,7890.0,9500.0,10926.0
6504294619,1.0,1.0,3.0,3.0,5.0,12.0,15.0,17.0,26.0,40.0,59.0,71.0,89.0
6504457668,107.0,193.0,247.0,337.0,428.0,493.0,578.0,678.0,744.0,827.0,927.0,1010.0,1066.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58856982900,31.0,38.0,64.0,82.0,114.0,177.0,257.0,359.0,499.0,654.0,849.0,1055.0,1325.0
58859605900,118.0,190.0,293.0,393.0,559.0,741.0,1044.0,1364.0,1760.0,10199.0,18783.0,23620.0,26509.0
59059234900,0.0,0.0,1.0,1.0,1.0,3.0,3.0,4.0,4.0,5.0,8.0,8.0,8.0


In [185]:
ov_yearly_citation_df2 = ov_yearly_citation_df1[ov_yearly_citation_df1.index.isin(target_authids_set)]
ov_yearly_citation_df2

,policy_citation_2011,policy_citation_2012,policy_citation_2013,policy_citation_2014,policy_citation_2015,policy_citation_2016,policy_citation_2017,policy_citation_2018,policy_citation_2019,policy_citation_2020,policy_citation_2021,policy_citation_2022,policy_citation_2023
authids,,,,,,,,,,,,,
6503956214,8,10,20,28,28,38,39,67,73,79,85,91,93
6503980107,0,0,0,0,0,1,1,1,1,1,1,1,1
6504250744,0,6,22,63,84,106,114,151,160,187,276,434,495
6504457668,0,1,1,3,7,16,16,16,16,16,23,32,32
6505812763,0,0,0,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58590644000,0,0,3,5,5,5,6,7,7,7,7,7,7
58711354400,0,0,0,0,0,0,0,0,0,0,4,5,5
58856982900,2,2,2,4,4,8,8,8,8,8,10,11,12


In [186]:
target_info_data = pd.merge(target_author_paperlist, target_author_num_cited, left_index=True, right_index=True, how='left')

In [187]:
target_info_data = pd.merge(target_info_data, ov_yearly_citation_df2, left_index=True, right_index=True, how='left')

In [188]:
target_info_data = target_info_data.fillna(0)

In [189]:
target_info_data

,paper_num_2015,academic_citation_2011,academic_citation_2012,academic_citation_2013,academic_citation_2014,academic_citation_2015,academic_citation_2016,academic_citation_2017,academic_citation_2018,academic_citation_2019,...,policy_citation_2014,policy_citation_2015,policy_citation_2016,policy_citation_2017,policy_citation_2018,policy_citation_2019,policy_citation_2020,policy_citation_2021,policy_citation_2022,policy_citation_2023
authids,,,,,,,,,,,,,,,,,,,,,
11140076200,38.0,836.0,1017.0,1196.0,1360.0,1580.0,1831.0,2080.0,2407.0,2746.0,...,4.0,4.0,7.0,8.0,11.0,12.0,15.0,17.0,17.0,19.0
8858824900,32.0,168.0,221.0,287.0,335.0,407.0,480.0,572.0,663.0,807.0,...,0.0,0.0,0.0,0.0,2.0,9.0,9.0,10.0,12.0,12.0
55835033800,4.0,0.0,0.0,1.0,10.0,20.0,40.0,57.0,76.0,91.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21743832000,27.0,172.0,243.0,314.0,365.0,451.0,511.0,564.0,623.0,666.0,...,15.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0,17.0
36642045300,50.0,120.0,260.0,399.0,565.0,756.0,1063.0,1361.0,1686.0,2075.0,...,4.0,10.0,10.0,11.0,20.0,24.0,35.0,49.0,55.0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23487676500,114.0,1316.0,1586.0,1918.0,2439.0,3107.0,4067.0,5075.0,6527.0,8262.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
23487806800,53.0,240.0,372.0,534.0,734.0,997.0,1380.0,1738.0,2112.0,2483.0,...,16.0,16.0,20.0,20.0,25.0,29.0,36.0,44.0,48.0,49.0
56371163300,93.0,908.0,1130.0,1367.0,1657.0,1972.0,2306.0,2693.0,3141.0,3552.0,...,7.0,9.0,10.0,15.0,17.0,25.0,35.0,44.0,57.0,74.0


In [190]:
# target_info_data.to_pickle('/disks/qnap3/users/21-tomokiyo/masters_thesis/data/guideline_paper/all_guideline_matching/candidate_feature_value.pickle')